In [2]:
import pandas as pd
import numpy as np
import pymc as pm

C:\Users\can2hr\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\can2hr\Anaconda3\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


## Speed
Our response will be predicted get off time
    - Need to decide how we want to do this
    - Option 1 is amount of time to get to a certain distance (Say 1.5 yards)
    - Option 2 would be predicted yards given a cetain amount of time
For predictors:
    - Height/weight of the tackle
    - Quarter
    - Whether it's an obvious passing situation
    - Left vs right tackle?

## Power
Our response here is some variation of percentage of the distance to the QB given up once engaged with the defensive end
For predictors:
    - Height/weight of both the tackle and the end
    - Obvious passing situation
    - Time to throw for the QB
    - Quarter

In [3]:
model_data = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/clustered.csv')

In [12]:
model_data

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,End_dir,End_pff_role,End_pff_positionLinedUp,End_pff_nflIdBlockedPlayer,End_pff_blockType,End_new_x,End_new_y,End_Flip,tackle_end_dist,tackle_end_facing
0,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,6,...,316.78,Pass Rush,LILB,NaN,NaN,2.12,-1.99,1.0,1.781376,42.63
1,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,7,...,302.16,Pass Rush,LILB,NaN,NaN,2.06,-1.96,1.0,1.714934,41.97
2,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,8,...,296.30,Pass Rush,LILB,NaN,NaN,1.96,-1.91,1.0,1.603777,37.27
3,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,9,...,292.78,Pass Rush,LILB,NaN,NaN,1.80,-1.85,1.0,1.448102,40.68
4,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,10,...,295.21,Pass Rush,LILB,NaN,NaN,1.56,-1.72,1.0,1.372953,42.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288402,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,25,...,259.09,Pass Rush,LE,NaN,NaN,2.61,-4.77,1.0,1.081896,3.64
288403,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,26,...,263.16,Pass Rush,LE,NaN,NaN,2.89,-4.81,1.0,1.115213,4.99
288404,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,27,...,269.23,Pass Rush,LE,NaN,NaN,3.14,-4.81,1.0,1.146211,16.80
288405,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,28,...,279.93,Pass Rush,LE,NaN,NaN,3.32,-4.78,1.0,1.196202,31.87


In [15]:
model_data2 = model_data[['uniqueplayId','QB_Flip','rushType','Power_prob','Speed_prob',
            'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'event','play_length_frames','QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp','QB_new_x', 'QB_new_y',
            'ball_new_x', 'ball_new_y', 'ball_Flip',
       'Tackle_nflId', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLinedUp',
       'Tackle_pff_nflIdBlockedPlayer', 'Tackle_pff_blockType', 'Tackle_new_x',
       'Tackle_new_y', 'Tackle_Flip', 'End_nflId', 'End_s', 'End_a', 'End_dis', 'End_o',
       'End_dir', 'End_pff_role', 'End_pff_positionLinedUp', 'End_new_x',
       'End_new_y', 'End_Flip', 'tackle_end_dist', 'tackle_end_facing']]

In [13]:
players = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv')

In [16]:
players = players[['height', 'weight', 'nflId','displayName']]

In [25]:
model_data3 = model_data2.merge(players, left_on = 'Tackle_nflId', right_on = 'nflId', how = 'left')

In [30]:
model_data4 = model_data3.rename(columns = {'height':'tackle_height','weight':'tackle_weight','displayName':'tackle_name'}).drop('nflId', axis = 1)

In [31]:
model_data5 = model_data4.merge(players, left_on = 'End_nflId', right_on = 'nflId', how = 'left')

In [33]:
model_data6 = model_data5.rename(columns = {'height':'end_height','weight':'end_weight','displayName':'end_name'}).drop('nflId', axis = 1)

In [108]:
plays = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv')

In [109]:
plays['obvious_pass'] = ((plays['down'] == 2) & (plays['yardsToGo'] > 10)) | ((plays['down'] == 3) & (plays['yardsToGo'] > 4)) | ((plays['down'] == 4) & (plays['yardsToGo'] > 2))

In [110]:
plays['obvious_pass'] = plays['obvious_pass'].astype(int)

In [111]:
plays = plays[['gameId','playId','quarter','obvious_pass']]

In [112]:
plays

,gameId,playId,quarter,obvious_pass
0,2021090900,97,1,0
1,2021090900,137,1,0
2,2021090900,187,1,0
3,2021090900,282,1,0
4,2021090900,349,1,1
...,...,...,...,...
8553,2021110100,4310,4,1
8554,2021110100,4363,4,0
8555,2021110100,4392,4,0
8556,2021110100,4411,4,1


In [113]:
model_data7 = model_data6.merge(plays, on = ['gameId','playId'], how = 'left')

In [114]:
model_data7

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,tackle_end_dist,tackle_end_facing,tackle_height,tackle_weight,tackle_name,end_height,end_weight,end_name,quarter,obvious_pass
0,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,6,...,1.781376,42.63,6-5,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0
1,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,7,...,1.714934,41.97,6-5,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0
2,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,8,...,1.603777,37.27,6-5,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0
3,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,9,...,1.448102,40.68,6-5,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0
4,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,10,...,1.372953,42.45,6-5,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288402,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,25,...,1.081896,3.64,6-6,325,Bobby Massie,6-4,315,Jordan Elliott,3,0
288403,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,26,...,1.115213,4.99,6-6,325,Bobby Massie,6-4,315,Jordan Elliott,3,0
288404,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,27,...,1.146211,16.80,6-6,325,Bobby Massie,6-4,315,Jordan Elliott,3,0
288405,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,28,...,1.196202,31.87,6-6,325,Bobby Massie,6-4,315,Jordan Elliott,3,0
